In [21]:
import numpy as np
import xarray as xr
import pandas as pd
from xgboost import XGBRegressor
import plotly.express as px
from functions import get_train_test_dataset, pinball

In [22]:
df_train, df_test = get_train_test_dataset()

### Corr Matrix

In [4]:
corr = df_train.corr()
corr.style.background_gradient(cmap='coolwarm')
# 'RdBu_r', 'BrBG_r', & PuOr_r are other good diverging colormaps

,reference_time,valid_time,CloudCover,SolarDownwardRadiation,Temperature_x,RelativeHumidity,Temperature_y,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,dtm,Wind_MWh_credit,Solar_MWh_credit,Total_MWh_credit,forcast_hours,year,month,day,hour
reference_time,1.000000,0.999987,-0.050967,0.057233,0.076194,-0.020111,0.052155,0.019915,0.023999,-0.026353,-0.012483,0.999987,0.010992,0.070854,0.056958,-0.000000,0.938939,-0.154190,-0.005615,0.000008
valid_time,0.999987,1.000000,-0.051012,0.057249,0.076211,-0.020059,0.052148,0.019893,0.023967,-0.026368,-0.012486,1.000000,0.011006,0.070856,0.056971,0.005181,0.938954,-0.154199,-0.005597,0.000057
CloudCover,-0.050967,-0.051012,1.000000,-0.093715,0.126572,0.228094,0.059239,-0.092034,-0.089922,0.097285,0.115272,-0.051012,0.123023,-0.058070,0.060032,-0.008719,-0.079670,0.100615,-0.011403,-0.035950
SolarDownwardRadiation,0.057233,0.057249,-0.093715,1.000000,0.436752,0.048432,0.165067,-0.023601,-0.022109,-0.200984,-0.185692,0.057249,-0.182620,0.893834,0.470495,0.003229,0.072950,-0.062005,-0.005240,-0.065524
Temperature_x,0.076194,0.076211,0.126572,0.436752,1.000000,0.381530,0.839616,-0.097245,-0.085732,-0.259717,-0.155402,0.076211,-0.137797,0.412432,0.173053,0.003515,-0.026746,0.271576,0.026969,0.136212
RelativeHumidity,-0.020111,-0.020059,0.228094,0.048432,0.381530,1.000000,0.265931,-0.069035,-0.057684,-0.186660,-0.059242,-0.020059,-0.045503,0.037145,-0.012363,0.009935,-0.022114,0.010161,0.043711,-0.046829
Temperature_y,0.052155,0.052148,0.059239,0.165067,0.839616,0.265931,1.000000,-0.026394,-0.020823,-0.215292,-0.157423,0.052148,-0.138539,0.118961,-0.030890,-0.001186,-0.129504,0.504564,0.046677,0.023025
WindDirection,0.019915,0.019893,-0.092034,-0.023601,-0.097245,-0.069035,-0.026394,1.000000,0.988472,0.178489,0.163085,0.019893,0.125135,-0.005450,0.099252,-0.004270,0.001812,0.045397,-0.005571,-0.027657
WindDirection:100,0.023999,0.023967,-0.089922,-0.022109,-0.085732,-0.057684,-0.020823,0.988472,1.000000,0.170889,0.160788,0.023967,0.123401,-0.002831,0.099642,-0.006066,0.006769,0.041594,-0.006594,-0.027220
WindSpeed,-0.026353,-0.026368,0.097285,-0.200984,-0.259717,-0.186660,-0.215292,0.178489,0.170889,1.000000,0.973103,-0.026368,0.764624,-0.158150,0.518373,-0.003050,-0.030340,0.018104,-0.000137,-0.013865


### Train Model

In [16]:
label = "Wind_MWh_credit"
# label = "Solar_MWh_credit"
# label = "to_MWh_credit"
columns = ["forcast_hours", "year", "month", "day", "hour", 'CloudCover', 'SolarDownwardRadiation', 'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100']

index = df_train[df_train[label].isna()].index
x = df_train.drop(index)[columns].to_numpy()
y = df_train.drop(index)[label].to_numpy()

index_test = df_test[df_test[label].isna()].index
x_test = df_test.drop(index_test)[columns].to_numpy()
y_test = df_test.drop(index_test)[label].to_numpy()

Modell

In [4]:
model = XGBRegressor()
model.load_model("wind_14-10-24.json")

In [6]:
for k, v in model.get_params().items():
    if v:
        print(f"{k} = {v}")

objective = reg:quantileerror
base_score = 2.93043E2
booster = gbtree
missing = nan


In [30]:
preds_train = model.predict(x)
preds = model.predict(x_test)

In [31]:
print(f"Shape Trainingsdaten: {preds_train.shape}\nShape Testdaten: {preds.shape}")

Shape Trainingsdaten: (921985, 9)
Shape Testdaten: (347499, 9)


In [35]:
scores_train = np.empty(9)
scores = np.empty(9)
for i in range(9):
    alpha = (i+1)/10
    scores_train[i] = pinball(y=y, q=preds_train[:,i], alpha=alpha).mean()
    scores[i] = pinball(y=y_test, q=preds[:,i], alpha=alpha).mean()    

In [38]:
scores_train

array([16.60457001, 26.4947575 , 32.38990064, 35.7459809 , 36.81013587,
       35.44323169, 30.30802435, 24.23303931, 15.16196457])

In [36]:
scores

array([34.38360562, 49.05626596, 56.74608883, 60.61485841, 59.34335773,
       54.66721445, 47.84519487, 37.34425551, 22.83023935])

In [39]:
scores_train.mean()

np.float64(28.132400539995267)

In [37]:
scores.mean()

np.float64(46.98123119124622)

### Split-Size

Versuche die Split-size herauszufinden

In [6]:
tree = model.get_booster().trees_to_dataframe()

In [14]:
help(model.get_booster())

Help on Booster in module xgboost.core object:

class Booster(builtins.object)
 |  Booster(params: Union[List, Dict[str, Any], NoneType] = None, cache: Optional[Sequence[xgboost.core.DMatrix]] = None, model_file: Union[ForwardRef('Booster'), bytearray, os.PathLike, str, NoneType] = None) -> None
 |
 |  A Booster of XGBoost.
 |
 |  Booster is the model of xgboost, that contains low level routines for
 |  training, prediction and evaluation.
 |
 |  Methods defined here:
 |
 |  __copy__(self) -> 'Booster'
 |
 |  __deepcopy__(self, _: Any) -> 'Booster'
 |      Return a copy of booster.
 |
 |  __del__(self) -> None
 |
 |  __getitem__(self, val: Union[int, numpy.integer, tuple, slice]) -> 'Booster'
 |      Get a slice of the tree-based model.
 |
 |      .. versionadded:: 1.3.0
 |
 |  __getstate__(self) -> Dict
 |      Helper for pickle.
 |
 |  __init__(self, params: Union[List, Dict[str, Any], NoneType] = None, cache: Optional[Sequence[xgboost.core.DMatrix]] = None, model_file: Union[Forward

In [10]:
tree.head(5)

,Tree,Node,ID,Feature,Split,Yes,No,Missing,Gain,Cover,Category
0,0,0,0-0,f13,9.362999,0-1,0-2,0-2,111645.875000,921985.0,NaN
1,0,1,0-1,f13,7.471886,0-3,0-4,0-4,6181.687500,493183.0,NaN
2,0,2,0-2,f13,11.124067,0-5,0-6,0-6,5453.863280,428802.0,NaN
3,0,3,0-3,f0,65.500000,0-7,0-8,0-8,939.250000,360336.0,NaN
4,0,4,0-4,f13,8.623632,0-9,0-10,0-10,768.953125,132847.0,NaN


In [23]:
tree.Tree.nunique()

9000

In [30]:
df = pd.DataFrame(columns=["Temp", "Regen", "Datum"], data=np.array([[11, 12, 13], [200, 300, 400], ["2023", "2024", "2025"]]).astype(float).T)


In [33]:
df.to_excel("test_excel2.xlsx")

### Data-Preprocessing

In [4]:
df = pd.concat([df_train, df_test])

In [5]:
df.isna().sum()

reference_time               0
valid_time                   0
CloudCover                   0
SolarDownwardRadiation       0
Temperature_x                0
RelativeHumidity             0
Temperature_y                0
WindDirection                0
WindDirection:100            0
WindSpeed                    0
WindSpeed:100                0
dtm                       1649
Wind_MWh_credit           3719
Solar_MWh_credit          2972
Total_MWh_credit          5042
forcast_hours                0
year                         0
month                        0
day                          0
hour                         0
dtype: int64

In [12]:
px.scatter(df[["dtm", "Wind_MWh_credit", "Solar_MWh_credit", "Total_MWh_credit"]].isna().join(df["reference_time"]).groupby("reference_time").sum())

duplicates

In [52]:
# Es gibt keine Duplikate
df[df.duplicated()]

,reference_time,valid_time,CloudCover,SolarDownwardRadiation,Temperature_x,RelativeHumidity,Temperature_y,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,dtm,Wind_MWh_credit,Solar_MWh_credit,Total_MWh_credit,forcast_hours,year,month,day,hour


dtm

In [25]:
xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240129_20240519.nc").to_dataframe().reset_index().reference_time.dt.tz_localize("UTC").max()

Timestamp('2024-05-19 00:00:00+0000', tz='UTC')

In [6]:
# Der Zeitraum der Messungen endet am 19.05.2024 23:30 Uhr -> Alle Wettervorhersagen danach sind nicht relevant und können gedropped werden
df = df.drop(df[df.valid_time >= "2024-05-20"].index).reset_index(drop=True)

In [7]:
df.isna().sum()

reference_time               0
valid_time                   0
CloudCover                   0
SolarDownwardRadiation       0
Temperature_x                0
RelativeHumidity             0
Temperature_y                0
WindDirection                0
WindDirection:100            0
WindSpeed                    0
WindSpeed:100                0
dtm                          0
Wind_MWh_credit           2070
Solar_MWh_credit          1323
Total_MWh_credit          3393
forcast_hours                0
year                         0
month                        0
day                          0
hour                         0
dtype: int64

Wind_MWh_credit

In [20]:
df[df.Wind_MWh_credit.isna() &  (df.Wind_MWh_credit.isna().shift(1) | df.Wind_MWh_credit.isna().shift(-1))].groupby("hour").size().sum()

np.int64(2068)

In [91]:
df["Wind_MWh_credit"].isna().nunique()
# df[["Wind_MWh_credit"]].isna().unique()

2

Solar_MWh_credit

In [14]:
df[df.Solar_MWh_credit.isna() & (df.Solar_MWh_credit.isna().shift(1) | df.Solar_MWh_credit.isna().shift(-1))].groupby("hour").size().sum()

np.int64(1163)

In [13]:
df[["Solar_MWh_credit"]].isna().join(df[["hour"]]).groupby("hour").sum()

,Solar_MWh_credit
hour,
0,0
1,0
2,60
3,80
4,40
5,60
6,41
7,140
8,160


Total_MWh_credit

In [15]:
df[df.Total_MWh_credit.isna() & (df.Total_MWh_credit.isna().shift(1) | df.Total_MWh_credit.isna().shift(-1))].groupby("hour").size().sum()

np.int64(3231)

### Modify the Data

In [38]:
df.columns

Index(['reference_time', 'valid_time', 'CloudCover', 'SolarDownwardRadiation',
       'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection',
       'WindDirection:100', 'WindSpeed', 'WindSpeed:100', 'dtm',
       'Wind_MWh_credit', 'Solar_MWh_credit', 'Total_MWh_credit',
       'forcast_hours', 'year', 'month', 'day', 'hour'],
      dtype='object')

In [41]:
columns = ['SolarDownwardRadiation', 'Temperature_x', 'Temperature_y', 'WindSpeed', 'WindSpeed:100']
n = round(len(df) / 100)

In [55]:
indexes = set()
for col in columns:
    indexes.update(set(df[col].nlargest(n).index))

In [58]:
len(indexes) / len(columns) / n

0.7445580371185907

In [61]:
df.drop(indexes).reset_index(drop=True)

,reference_time,valid_time,CloudCover,SolarDownwardRadiation,Temperature_x,RelativeHumidity,Temperature_y,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,dtm,Wind_MWh_credit,Solar_MWh_credit,Total_MWh_credit,forcast_hours,year,month,day,hour
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,0.450405,0.000000,13.646173,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604,2020-09-20 00:00:00+00:00,498.142,0.0,498.142,0.0,2020,9,20,0
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,0.472211,0.000000,13.658508,85.012253,15.412510,61.203667,61.726974,9.905537,11.648819,2020-09-20 00:30:00+00:00,478.788,0.0,478.788,0.5,2020,9,20,0
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,0.494018,0.000000,13.670843,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033,2020-09-20 01:00:00+00:00,470.522,0.0,470.522,1.0,2020,9,20,1
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,0.520214,0.000000,13.732101,84.357880,15.451218,60.511028,61.111038,9.631039,11.354128,2020-09-20 01:30:00+00:00,482.183,0.0,482.183,1.5,2020,9,20,1
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,0.546410,0.000000,13.793360,83.904999,15.494086,60.202801,60.853306,9.494630,11.213223,2020-09-20 02:00:00+00:00,459.216,0.0,459.216,2.0,2020,9,20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224210,2024-05-19 00:00:00+00:00,2024-05-19 21:30:00+00:00,0.024549,0.010731,11.203708,91.634132,11.190109,21.214108,21.933590,6.882957,8.258531,2024-05-19 21:30:00+00:00,113.759,0.0,113.759,21.5,2024,5,19,21
1224211,2024-05-19 00:00:00+00:00,2024-05-19 22:00:00+00:00,0.027738,0.012567,10.840198,91.232635,11.127069,22.417616,22.716902,7.040593,8.282218,2024-05-19 22:00:00+00:00,98.545,0.0,98.545,22.0,2024,5,19,22
1224212,2024-05-19 00:00:00+00:00,2024-05-19 22:30:00+00:00,0.045711,-0.001822,10.559702,90.249336,11.237669,22.327656,22.829685,6.842084,8.272179,2024-05-19 22:30:00+00:00,85.653,0.0,85.653,22.5,2024,5,19,22
1224213,2024-05-19 00:00:00+00:00,2024-05-19 23:00:00+00:00,0.063685,-0.016211,10.279205,89.266037,11.348270,22.237694,22.942471,6.643576,8.262139,2024-05-19 23:00:00+00:00,81.582,0.0,81.582,23.0,2024,5,19,23


### Einlesen ung Auswerten von Ergebnisse

In [15]:
exl = pd.read_excel("Ergebnisse.xlsx")
exl.drop(columns=["Unnamed: 0"], inplace=True)

In [16]:
exl.columns

Index(['n_estimate', 'max_depth', 'eta', 'gamma', 'lambda', 'train_score',
       'test_score', 'Trainingszeit', 'Plattform', 'Spalten', 'Label'],
      dtype='object')

In [20]:
col = "lambda"
exl.loc[exl["Label"] == "Wind"][[col, "train_score", "test_score"]].groupby(col).aggregate(["mean", "size"])

train_score      test_score     
              mean size       mean size
lambda                                 
1        30.146127  142  47.478099  142
2        34.487500    8  45.125000    8
4        34.487500    8  45.125000    8
9        34.487500    8  45.125000    8

### Pinball-Score mit 2 Modellen zusammenführen

In [27]:
model_solar = XGBRegressor()
model_solar.load_model("solar_14-10-24.json")
model_wind = XGBRegressor()
model_wind.load_model("wind_14-10-24.json")
model_total = XGBRegressor()
model_total.load_model("total_14-10-24.json")

In [24]:
# label = "Wind_MWh_credit"
label = "Solar_MWh_credit"
# label = "to_MWh_credit"
columns = ["forcast_hours", "year", "month", "day", "hour", 'CloudCover', 'SolarDownwardRadiation', 'Temperature_x', 'RelativeHumidity', 'Temperature_y', 'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100']

index = df_train[df_train[label].isna()].index
x = df_train.drop(index)[columns].to_numpy()
y = df_train.drop(index)[label].to_numpy()

index_test = df_test[df_test[label].isna()].index
x_test = df_test.drop(index_test)[columns].to_numpy()
y_test = df_test.drop(index_test)[label].to_numpy()

In [29]:
preds_solar = model_solar.predict(x_test)
preds_wind = model_wind.predict(x_test)
preds_total = model_total.predict(x_test)

In [40]:
pinball_scores = np.empty((4, 9))
for i in range(9):
    alpha = (i+1)/10
    pinball_scores[0, i] = pinball(y=y_test, q=preds_solar[:,i], alpha=alpha).mean()
    pinball_scores[1, i] = pinball(y=y_test, q=preds_wind[:,i], alpha=alpha).mean()
    pinball_scores[2, i] = pinball(y=y_test, q=preds_solar[:,i] + preds_wind[:,i], alpha=alpha).mean()
    pinball_scores[3, i] = pinball(y=y_test, q=preds_total[:,i], alpha=alpha).mean()

In [47]:
print(f"Pinballscore Solar = {pinball_scores[0].mean().round(2)}")
print(f"Pinballscore Wind = {pinball_scores[1].mean().round(2)}")
print(f"Pinballscore Sum = {pinball_scores[2].mean().round(2)}")
print(f"Pinballscore Total = {pinball_scores[3].mean().round(2)}")

Pinballscore Solar = 12.84
Pinballscore Wind = 130.23
Pinballscore Sum = 130.67
Pinballscore Total = 133.63


### Datensatz für Colab vorbereiten

In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
df_train = pd.read_parquet("df_train.parquet")
df_test = pd.read_parquet("df_test.parquet")

In [3]:
windspeed_train_pca = df_train[["WindSpeed", "WindSpeed:100"]].to_numpy()
windspeed_test_pca = df_test[["WindSpeed", "WindSpeed:100"]].to_numpy()

scale_pca_pipe = Pipeline([
    ("scaling", StandardScaler()),
    ("pca", PCA(n_components=1))
])

windspeed_train_pca = scale_pca_pipe.fit_transform(windspeed_train_pca)
windspeed_test_pca = scale_pca_pipe.transform(windspeed_test_pca)

df_train["WindSpeedPCA"] = windspeed_train_pca
df_test["WindSpeedPCA"] = windspeed_test_pca

winddirection_train_pca = df_train[["WindDirection", "WindDirection:100"]].to_numpy()
winddirection_test_pca = df_test[["WindDirection", "WindDirection:100"]].to_numpy()

winddirection_train_pca = scale_pca_pipe.fit_transform(winddirection_train_pca)
winddirection_test_pca = scale_pca_pipe.transform(winddirection_test_pca)

df_train["WindDirectionPCA"] = winddirection_train_pca
df_test["WindDirectionPCA"] = winddirection_test_pca

In [4]:
df_train.to_parquet("df_train_pca.parquet")
df_test.to_parquet("df_test_pca.parquet")

In [47]:
df_test.drop(df_test["Wind_MWh_credit"].isna().index)

,reference_time,valid_time,RelativeHumidity,temp_hornsea,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,hours_after,CloudCover,...,temp_x_solar_interaction,temp_y_solar_interaction,wind_interaction,wind_interaction_100,humidity_wind_interaction,wind_gradient,CloudCover_lag_1h,cloud_cover_change,WindSpeedPCA,WindDirectionPCA


In [5]:
df_train.columns

Index(['reference_time', 'valid_time', 'RelativeHumidity', 'temp_hornsea',
       'WindDirection', 'WindDirection:100', 'WindSpeed', 'WindSpeed:100',
       'hours_after', 'CloudCover', 'SolarDownwardRadiation', 'temp_solar',
       'dtm', 'MIP', 'Solar_MW', 'Solar_capacity_mwp',
       'Solar_installedcapacity_mwp', 'Wind_MW', 'SS_Price', 'boa_MWh',
       'DA_Price', 'Wind_MWh_credit', 'Solar_MWh_credit', 'year', 'month',
       'day', 'hour', 'season', 'time_of_day', 'wind_dir_cat',
       'time_of_day_afternoon', 'time_of_day_morning', 'time_of_day_night',
       'season_autumn', 'season_spring', 'season_summer', 'season_winter',
       'wind_dir_cat_E', 'wind_dir_cat_N', 'wind_dir_cat_NE',
       'wind_dir_cat_NW', 'wind_dir_cat_S', 'wind_dir_cat_SE',
       'wind_dir_cat_SW', 'wind_dir_cat_W', 'adjusted_solar_radiation',
       'temp_x_solar_interaction', 'temp_y_solar_interaction',
       'wind_interaction', 'wind_interaction_100', 'humidity_wind_interaction',
       'wind_gradi

In [50]:
lab = "Wind_MWh_credit"
df_test[lab].dropna().index

Index([1022322, 1022323, 1022324, 1022325, 1022326, 1022327, 1022328, 1022329,
       1022330, 1022331,
       ...
       1110759, 1110760, 1110761, 1110762, 1110763, 1110764, 1110765, 1110766,
       1110767, 1110768],
      dtype='int64', length=87483)